## This is a file to test methods on how to gather data using the "Original dataset" csv file that is in this Data folder.

In [29]:
import pandas as pd
import requests
import json
import spacy


# Read Kaggle dataset, and extract the ISBN 13 for all the books

In [6]:
kaggle_data = pd.read_csv("books.csv")  # read the Kaggle books dataset
isbn_13 = kaggle_data['isbn13']         # extract the ISBN-13 column

isbn_13.shape 

(11127,)

In [40]:
test = isbn_13.head(10)
test

0    9780439785969
1    9780439358071
2    9780439554893
3    9780439655484
4    9780439682589
5    9780976540601
6    9780439827607
7    9780517226957
8    9780345453747
9    9781400052929
Name: isbn13, dtype: object

# Get the books from the Google Books API using the ISNB-13s 

In [41]:
# create an empty list to store the book data
book_data_list = []
url_base = 'https://www.googleapis.com/books/v1/volumes?q=isbn:'

# iterate over the ISBN 13 numbers, and get the information we need using the Google Books API
for i, isbn in enumerate(test):

    # generate the URL for the Google Books API search 
    url = url_base + isbn

    # get the book data; convert it to a dictionary using .json()
    book_data = requests.get(url).json()

    print(f'{i} {isbn}: {book_data.keys()}', end=': ')
    
    if 'totalItems' in book_data and book_data['totalItems'] > 0:

        print(book_data['totalItems'], end=': ')
        print(book_data['items'][0]['volumeInfo']['title'], end='')
        # extract the relevant fields and create a new row in the DataFrame
        book = {
            'title': book_data['items'][0]['volumeInfo'].get('title', ''),
            'subtitle': book_data['items'][0]['volumeInfo'].get('subtitle', ''),
            'authors': book_data['items'][0]['volumeInfo'].get('authors', ''),
            'publisher': book_data['items'][0]['volumeInfo'].get('publisher', ''),
            'publishdate': book_data['items'][0]['volumeInfo'].get('publishedDate', ''),
            'description': book_data['items'][0]['volumeInfo'].get('description', ''),
            'isbn_13': isbn,
            'page_count': book_data['items'][0]['volumeInfo'].get('pageCount', ''),
            'main_categories': book_data['items'][0]['volumeInfo'].get('categories', ''),
            'categories': book_data['items'][0]['volumeInfo'].get('categories', ''),
            'average_rating': book_data['items'][0]['volumeInfo'].get('averageRating', ''),
            'ratings_count': book_data['items'][0]['volumeInfo'].get('ratingsCount', ''),
            'maturity_rating': book_data['items'][0]['volumeInfo'].get('maturityRating', '')
        }

        # append the new dictionary to the book_data_list
        book_data_list.append(book)
    else:

        # handle the case where no results were returned for the ISBN-13 number
        # print(f"No results found for ISBN-13 {isbn}")
        pass
    
    print()

# convert the list of dictionaries to a DataFrame
response_df = pd.DataFrame(book_data_list)

0 9780439785969: dict_keys(['kind', 'totalItems']): 
1 9780439358071: dict_keys(['kind', 'totalItems']): 
2 9780439554893: dict_keys(['kind', 'totalItems', 'items']): 1: Harry Potter and the Chamber of Secrets
3 9780439655484: dict_keys(['kind', 'totalItems', 'items']): 1: Harry Potter and the Prisoner of Azkaban
4 9780439682589: dict_keys(['kind', 'totalItems', 'items']): 1: Harry Potter
5 9780976540601: dict_keys(['kind', 'totalItems', 'items']): 1: Unauthorized Harry Potter and the Deathly Hallows News
6 9780439827607: dict_keys(['kind', 'totalItems', 'items']): 2: Harry Potter and the Prisoner of Azkaban
7 9780517226957: dict_keys(['kind', 'totalItems', 'items']): 1: The Ultimate Hitchhiker's Guide
8 9780345453747: dict_keys(['kind', 'totalItems', 'items']): 1: The Ultimate Hitchhiker's Guide to the Galaxy
9 9781400052929: dict_keys(['kind', 'totalItems', 'items']): 2: The Hitchhiker's Guide to the Galaxy 25th Anniversary Edition


In [42]:
response_df

,title,subtitle,authors,publisher,publishdate,description,isbn_13,page_count,main_categories,categories,average_rating,ratings_count,maturity_rating
0,Harry Potter and the Chamber of Secrets,,"[J. K. Rowling, Mary GrandPre]",Arthur a Levine,2003,When the Chamber of Secrets is opened again at...,9780439554893,341,[Juvenile Fiction],[Juvenile Fiction],4.5,2273,NOT_MATURE
1,Harry Potter and the Prisoner of Azkaban,,[J. K. Rowling],Scholastic Paperbacks,2004,During his third year at Hogwarts School for W...,9780439655484,547,[Juvenile Fiction],[Juvenile Fiction],4.5,2122,NOT_MATURE
2,Harry Potter,"5 Years of Magic, Adventure, and Mystery at Ho...",[J. K. Rowling],,2004,,9780439682589,0,,,4.5,13,NOT_MATURE
3,Unauthorized Harry Potter and the Deathly Hall...,Harry Potter Book Seven and Half-Blood Prince ...,[W. Frederick Zimmerman],Nimble Books,2005-04,Through the magic of print-on-demand technolog...,9780976540601,152,[Fiction],[Fiction],3.5,11,NOT_MATURE
4,Harry Potter and the Prisoner of Azkaban,,[J. K. Rowling],,1999,"""During his third year at Hogwarts School for ...",9780439827607,435,[Children's stories],[Children's stories],4.5,16,NOT_MATURE
5,The Ultimate Hitchhiker's Guide,Five Complete Novels and One Story,[Douglas Adams],Gramercy,2005,6 Science fiction-romaner.,9780517226957,844,"[Dent, Arthur (Fictitious character)]","[Dent, Arthur (Fictitious character)]",4.5,35,NOT_MATURE
6,The Ultimate Hitchhiker's Guide to the Galaxy,,[Douglas Adams],Del Rey,2002-04-30,"In one complete volume, here are the five clas...",9780345453747,836,[Fiction],[Fiction],4.5,179,NOT_MATURE
7,The Hitchhiker's Guide to the Galaxy 25th Anni...,A Novel,[Douglas Adams],Crown,2004-08-03,NEW YORK TIMES BESTSELLER • “Extremely funny ....,9781400052929,0,[Fiction],[Fiction],,,NOT_MATURE


In [10]:
#commented out to not edit huge csv
#response_df.to_csv('isbn13_results.csv')

Using Google Books API to fix missing data in Kaggle Dataset

In [11]:
# get the faulty ISBN
isbn = isbn_13.loc[0]
print(isbn)

# url for the book
url = f"https://www.googleapis.com/books/v1/volumes?q=isbn:{isbn}"

# get the book
response = requests.get(url)

# convert the HTTP Response object to a dict
json = response.json()

# get the keys for the dict
json.keys()

9780439785969


dict_keys(['kind', 'totalItems'])

# testing SpaCy

### 1 and 1 

In [39]:
# Load English tokenizer, tagger, parser and NER
nlp = spacy.load("en_core_web_lg")

# Process whole documents
book_d_1 = nlp(response_df["description"][0])
book_d_2 = nlp(response_df["description"][1])

book_d_1.similarity(book_d_2)

0.8970893140160056

### 1 and many

In [47]:
# Load LARGE English tokenizer, tagger, parser and NER
nlp = spacy.load("en_core_web_lg")

# pulling and creating variables for books descriptions
book_d_1 = nlp(response_df["description"][0])
book_d_2 = nlp(response_df["description"][1])
book_d_3 = nlp(response_df["description"][2])
book_d_4 = nlp(response_df["description"][3])
book_d_5 = nlp(response_df["description"][4])
book_d_6 = nlp(response_df["description"][5])

# Descriptions from books
texts = [book_d_2, book_d_3, book_d_4, book_d_5, book_d_6]

# Create a list of Doc objects
docs = [nlp(text) for text in texts]

# Compare doc1 with each of the docs in the list
for doc in docs:
    similarity_score = book_d_1.similarity(doc)
    head_text = doc.text[:5]  # Get the first 5 characters of the text
    print(f"Similarity score with '{head_text}': {similarity_score}")

Similarity score with 'Durin': 0.8970893140160056
Similarity score with '': 0.0
Similarity score with 'Throu': 0.8815107060053393
Similarity score with '"Duri': 0.8823296614181213
Similarity score with '6 Sci': 0.4377405652863445


C:\Users\danie\AppData\Local\Temp\ipykernel_42924\3463707688.py:20: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity_score = book_d_1.similarity(doc)
